# Comparison of TSS classifier Performance of multiple Datasets
In this notebook, the classifier trained in Model_Training is tested on multiple Datasets

In [1]:
import sys
sys.path.append('../')

In [2]:
from app.job import JobObject as jo

In [3]:
from app import TSSclassifier as cs

In [4]:
def jaccard(common, a, b):
    return(common/(a + b - common))

In [5]:
def precision(common, a):
    if(a != 0):
        return (common/a)
    else:
        return 0

In [6]:
def recall(common, b):
    if(b != 0):
        return (common/b)
    else:
        return 0

In [7]:
import glob
import os

In [8]:
prediction_time = []

In [9]:
sample_size = []

In [10]:
import timeit

# Bacteroides

In [11]:
conditions = ["ELP", "MLP", "Stat"]

In [12]:
directory = "Data/Subset_OnlyChrom/"

In [13]:
pattern_forward = os.path.join(directory, '*forward*')
pattern_reverse = os.path.join(directory, '*reverse*')

In [14]:
files_forward = glob.glob(pattern_forward)
files_reverse = glob.glob(pattern_reverse)

df_fordward = {}

for condition in conditions:
    file_list = []
    for file_path in files_forward:
            if(condition in str(file_path)):
                file_list += [file_path]
    df_fordward[condition] = file_list

df_reverse = {}

for condition in conditions:
    file_list = []
    for file_path in files_reverse:
            if(condition in str(file_path)):
                file_list += [file_path]
    df_reverse[condition] = file_list

In [15]:
tss_by_type = {}

for condition in conditions:

    start_time = timeit.default_timer()

    if(condition == "Stat"):
    
        forward_object = jo.JobObject(filepaths=df_fordward[condition], name="forward", condition_name="static", 
                                      master_table_path="Data/Subset_OnlyChrom/MasterTable_chrom.tsv", gff_path= "Data/Subset_OnlyChrom/NC_004663.gff", is_reverse_strand = False)
        reverse_object = jo.JobObject(filepaths=df_reverse[condition], name="reverse", condition_name="static", 
                                      master_table_path="Data/Subset_OnlyChrom/MasterTable_chrom.tsv", gff_path= "Data/Subset_OnlyChrom/NC_004663.gff", is_reverse_strand = True)
    else:
        forward_object = jo.JobObject(filepaths=df_fordward[condition], name="forward", condition_name=condition, 
                                      master_table_path="Data/Subset_OnlyChrom/MasterTable_chrom.tsv", gff_path= "Data/Subset_OnlyChrom/NC_004663.gff", is_reverse_strand = False)
        reverse_object = jo.JobObject(filepaths=df_reverse[condition], name="reverse", condition_name=condition, 
                                      master_table_path="Data/Subset_OnlyChrom/MasterTable_chrom.tsv", gff_path= "Data/Subset_OnlyChrom/NC_004663.gff", is_reverse_strand = True)

    forward_object.process()
    reverse_object.process()

    elapsed = timeit.default_timer() - start_time

    prediction_time += [elapsed]

    dataset_size_forward = forward_object.processedDF.index.values[-1]
    dataset_size_reverse = reverse_object.processedDF.index.values[-1]

    dataset_size = max(dataset_size_forward, dataset_size_reverse)

    sample_size += [dataset_size]
    
    masterTable_forward = forward_object.master_table
    common_forward = forward_object.common_tss
    classifed_forward = forward_object.classified_tss

    masterTable_reverse = reverse_object.master_table
    common_reverse = reverse_object.common_tss
    classifed_reverse = reverse_object.classified_tss

    for tss_type in cs.TSSType:
        master_table_filtered_forward = masterTable_forward[masterTable_forward["TSS type"] == tss_type.value]
        master_table_filtered_reverse = masterTable_reverse[masterTable_reverse["TSS type"] == tss_type.value]

        classified_filtered_forward = classifed_forward[classifed_forward["TSS type"] == tss_type.value]
        classified_filtered_reverse = classifed_reverse[classifed_reverse["TSS type"] == tss_type.value]

        common_filtered_forward = common_forward[common_forward["TSS type"] == tss_type.value]
        common_filtered_reverse = common_reverse[common_reverse["TSS type"] == tss_type.value]
        
        if(tss_type.value not in tss_by_type.keys()):
            save_dict = {}
            save_dict["master_table"] = list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            save_dict["common"] = list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            save_dict["classified"] = list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value] = save_dict
        else:
            tss_by_type[tss_type.value]["master_table"] += list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["common"] += list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["classified"] += list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)
        

In [16]:
performance_dict = {}
for tss_type in cs.TSSType:
    tss_by_type[tss_type.value]["master_table"] = set(tss_by_type[tss_type.value]["master_table"])
    tss_by_type[tss_type.value]["common"] = set(tss_by_type[tss_type.value]["common"])
    tss_by_type[tss_type.value]["classified"] = set(tss_by_type[tss_type.value]["classified"])

    common = len(tss_by_type[tss_type.value]["common"])
    classified = len(tss_by_type[tss_type.value]["classified"])
    master_table = len(tss_by_type[tss_type.value]["master_table"])
    
    performance_dict[tss_type.value] = {"precision": precision(common, classified), "recall": recall(common, master_table), "jaccard": jaccard(common, classified, master_table)}

In [17]:
performances_full = {}

In [18]:
performances_full["bacteroides chromosome"] = performance_dict

# S.aureus

In [19]:
conditions = ["WT", "Rny3"]

In [20]:
directory = "Data/Saureus/files"

In [21]:
pattern_forward = os.path.join(directory, '*forward*')
pattern_reverse = os.path.join(directory, '*reverse*')

In [22]:
files_forward = glob.glob(pattern_forward)
files_reverse = glob.glob(pattern_reverse)

df_fordward = {}

for condition in conditions:
    file_list = []
    for file_path in files_forward:
        if(condition == "WT"):
            if(condition in str(file_path)):
                file_list += [file_path]
        else:
            if("rny" in str(file_path)):
                file_list += [file_path]
    df_fordward[condition] = file_list

df_reverse = {}

for condition in conditions:
    file_list = []
    for file_path in files_reverse:
        if(condition == "WT"):
            if(condition in str(file_path)):
                file_list += [file_path]
        else:
            if("rny" in str(file_path)):
                file_list += [file_path]
    df_reverse[condition] = file_list

In [23]:
tss_by_type = {}

for condition in conditions:

    start_time = timeit.default_timer()
   
    forward_object = jo.JobObject(filepaths=df_fordward[condition], name="forward", condition_name=condition, 
                                      master_table_path="Data/Saureus/MasterTable.tsv", gff_path= "Data/Saureus/files/NC_009641.gff", is_reverse_strand = False)
    reverse_object = jo.JobObject(filepaths=df_reverse[condition], name="reverse", condition_name=condition, 
                                      master_table_path="Data/Saureus/MasterTable.tsv", gff_path= "Data/Saureus/files/NC_009641.gff", is_reverse_strand = True)

    forward_object.process()
    reverse_object.process()

    elapsed = timeit.default_timer() - start_time
    prediction_time += [elapsed]

    dataset_size_forward = forward_object.processedDF.index.values[-1]
    dataset_size_reverse = reverse_object.processedDF.index.values[-1]

    dataset_size = max(dataset_size_forward, dataset_size_reverse)

    sample_size += [dataset_size]
    
    masterTable_forward = forward_object.master_table
    common_forward = forward_object.common_tss
    classifed_forward = forward_object.classified_tss

    masterTable_reverse = reverse_object.master_table
    common_reverse = reverse_object.common_tss
    classifed_reverse = reverse_object.classified_tss

    for tss_type in cs.TSSType:
        master_table_filtered_forward = masterTable_forward[masterTable_forward["TSS type"] == tss_type.value]
        master_table_filtered_reverse = masterTable_reverse[masterTable_reverse["TSS type"] == tss_type.value]

        classified_filtered_forward = classifed_forward[classifed_forward["TSS type"] == tss_type.value]
        classified_filtered_reverse = classifed_reverse[classifed_reverse["TSS type"] == tss_type.value]

        common_filtered_forward = common_forward[common_forward["TSS type"] == tss_type.value]
        common_filtered_reverse = common_reverse[common_reverse["TSS type"] == tss_type.value]
        
        if(tss_type.value not in tss_by_type.keys()):
            save_dict = {}
            save_dict["master_table"] = list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            save_dict["common"] = list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            save_dict["classified"] = list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value] = save_dict
        else:
            tss_by_type[tss_type.value]["master_table"] += list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["common"] += list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["classified"] += list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)

In [24]:
performance_dict = {}
for tss_type in cs.TSSType:
    tss_by_type[tss_type.value]["master_table"] = set(tss_by_type[tss_type.value]["master_table"])
    tss_by_type[tss_type.value]["common"] = set(tss_by_type[tss_type.value]["common"])
    tss_by_type[tss_type.value]["classified"] = set(tss_by_type[tss_type.value]["classified"])

    common = len(tss_by_type[tss_type.value]["common"])
    classified = len(tss_by_type[tss_type.value]["classified"])
    master_table = len(tss_by_type[tss_type.value]["master_table"])
    
    performance_dict[tss_type.value] = {"precision": precision(common, classified), "recall": recall(common, master_table), "jaccard": jaccard(common, classified, master_table)}

In [25]:
performances_full["S.aureus"] = performance_dict

# Pseudomonas 

In [26]:
conditions = ["infected", "uninfected"]

In [27]:
directory = "Data/Pseudomonas/files"

In [28]:
pattern_forward = os.path.join(directory, '*forward*')
pattern_reverse = os.path.join(directory, '*reverse*')

In [29]:
files_forward = glob.glob(pattern_forward)
files_reverse = glob.glob(pattern_reverse)

df_fordward = {}

for condition in conditions:
    file_list = []
    for file_path in files_forward:
        if(condition == "infected"):
            if(condition in str(file_path) and "uninfected" not in str(file_path)):
                file_list += [file_path]
        else:
            if(condition in str(file_path)):
                file_list += [file_path]
    df_fordward[condition] = file_list

df_reverse = {}

for condition in conditions:
    file_list = []
    for file_path in files_reverse:
        if(condition == "infected"):
            if(condition in str(file_path) and "uninfected" not in str(file_path)):
                file_list += [file_path]
        else:
            if(condition in str(file_path)):
                file_list += [file_path]
    df_reverse[condition] = file_list

In [ ]:
tss_by_type = {}

for condition in conditions:

    start_time = timeit.default_timer()

    forward_object = jo.JobObject(filepaths=df_fordward[condition], name="forward", condition_name=condition, 
                                      master_table_path="Data/Pseudomonas/MasterTable.tsv", gff_path= "Data/Pseudomonas/files/PAO1_annotation.gff", is_reverse_strand = False)
    reverse_object = jo.JobObject(filepaths=df_reverse[condition], name="reverse", condition_name=condition, 
                                      master_table_path="Data/Pseudomonas/MasterTable.tsv", gff_path= "Data/Pseudomonas/files/PAO1_annotation.gff", is_reverse_strand = True)

    forward_object.process()
    reverse_object.process()

    elapsed = timeit.default_timer() - start_time
    prediction_time += [elapsed]

    dataset_size_forward = forward_object.processedDF.index.values[-1]
    dataset_size_reverse = reverse_object.processedDF.index.values[-1]

    dataset_size = max(dataset_size_forward, dataset_size_reverse)

    sample_size += [dataset_size]
    
    masterTable_forward = forward_object.master_table
    common_forward = forward_object.common_tss
    classifed_forward = forward_object.classified_tss

    masterTable_reverse = reverse_object.master_table
    common_reverse = reverse_object.common_tss
    classifed_reverse = reverse_object.classified_tss

    for tss_type in cs.TSSType:
        master_table_filtered_forward = masterTable_forward[masterTable_forward["TSS type"] == tss_type.value]
        master_table_filtered_reverse = masterTable_reverse[masterTable_reverse["TSS type"] == tss_type.value]

        classified_filtered_forward = classifed_forward[classifed_forward["TSS type"] == tss_type.value]
        classified_filtered_reverse = classifed_reverse[classifed_reverse["TSS type"] == tss_type.value]

        common_filtered_forward = common_forward[common_forward["TSS type"] == tss_type.value]
        common_filtered_reverse = common_reverse[common_reverse["TSS type"] == tss_type.value]
        
        if(tss_type.value not in tss_by_type.keys()):
            save_dict = {}
            save_dict["master_table"] = list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            save_dict["common"] = list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            save_dict["classified"] = list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value] = save_dict
        else:
            tss_by_type[tss_type.value]["master_table"] += list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["common"] += list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["classified"] += list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)

In [ ]:
performance_dict = {}
for tss_type in cs.TSSType:
    tss_by_type[tss_type.value]["master_table"] = set(tss_by_type[tss_type.value]["master_table"])
    tss_by_type[tss_type.value]["common"] = set(tss_by_type[tss_type.value]["common"])
    tss_by_type[tss_type.value]["classified"] = set(tss_by_type[tss_type.value]["classified"])

    common = len(tss_by_type[tss_type.value]["common"])
    classified = len(tss_by_type[tss_type.value]["classified"])
    master_table = len(tss_by_type[tss_type.value]["master_table"])
    
    performance_dict[tss_type.value] = {"precision": precision(common, classified), "recall": recall(common, master_table), "jaccard": jaccard(common, classified, master_table)}

In [ ]:
performances_full["pseudomonas"] = performance_dict

# Bacteroides Plasmid

In [ ]:
conditions = ["ELP", "MLP", "Stat"]

In [ ]:
directory = "Data/Subset_OnlyPlasmid/"

In [ ]:
pattern_forward = os.path.join(directory, '*forward*')
pattern_reverse = os.path.join(directory, '*reverse*')

In [ ]:
files_forward = glob.glob(pattern_forward)
files_reverse = glob.glob(pattern_reverse)

df_fordward = {}

for condition in conditions:
    file_list = []
    for file_path in files_forward:
            if(condition in str(file_path)):
                file_list += [file_path]
    df_fordward[condition] = file_list

df_reverse = {}

for condition in conditions:
    file_list = []
    for file_path in files_reverse:
            if(condition in str(file_path)):
                file_list += [file_path]
    df_reverse[condition] = file_list

In [ ]:
tss_by_type = {}

for condition in conditions:

    start_time = timeit.default_timer()

    if(condition == "Stat"):
    
        forward_object = jo.JobObject(filepaths=df_fordward[condition], name="forward", condition_name="static", 
                                      master_table_path="Data/Subset_OnlyPlasmid/MasterTable_plasmid.tsv", gff_path= "Data/Subset_OnlyPlasmid/NC_004703.gff", is_reverse_strand = False)
        reverse_object = jo.JobObject(filepaths=df_reverse[condition], name="reverse", condition_name="static", 
                                      master_table_path="Data/Subset_OnlyPlasmid/MasterTable_plasmid.tsv", gff_path= "Data/Subset_OnlyPlasmid/NC_004703.gff", is_reverse_strand = True)
    else:
        forward_object = jo.JobObject(filepaths=df_fordward[condition], name="forward", condition_name=condition, 
                                      master_table_path="Data/Subset_OnlyPlasmid/MasterTable_plasmid.tsv", gff_path= "Data/Subset_OnlyPlasmid/NC_004703.gff", is_reverse_strand = False)
        reverse_object = jo.JobObject(filepaths=df_reverse[condition], name="reverse", condition_name=condition, 
                                      master_table_path="Data/Subset_OnlyPlasmid/MasterTable_plasmid.tsv", gff_path= "Data/Subset_OnlyPlasmid/NC_004703.gff", is_reverse_strand = True)

    forward_object.process()
    reverse_object.process()

    elapsed = timeit.default_timer() - start_time
    prediction_time += [elapsed]

    dataset_size_forward = forward_object.processedDF.index.values[-1]
    dataset_size_reverse = reverse_object.processedDF.index.values[-1]

    dataset_size = max(dataset_size_forward, dataset_size_reverse)

    sample_size += [dataset_size]
    
    masterTable_forward = forward_object.master_table
    common_forward = forward_object.common_tss
    classifed_forward = forward_object.classified_tss

    masterTable_reverse = reverse_object.master_table
    common_reverse = reverse_object.common_tss
    classifed_reverse = reverse_object.classified_tss

    print(classifed_forward)
    print(classifed_reverse)

    for tss_type in cs.TSSType:
        master_table_filtered_forward = masterTable_forward[masterTable_forward["TSS type"] == tss_type.value]
        master_table_filtered_reverse = masterTable_reverse[masterTable_reverse["TSS type"] == tss_type.value]

        classified_filtered_forward = classifed_forward[classifed_forward["TSS type"] == tss_type.value]
        classified_filtered_reverse = classifed_reverse[classifed_reverse["TSS type"] == tss_type.value]

        common_filtered_forward = common_forward[common_forward["TSS type"] == tss_type.value]
        common_filtered_reverse = common_reverse[common_reverse["TSS type"] == tss_type.value]
        
        if(tss_type.value not in tss_by_type.keys()):
            save_dict = {}
            save_dict["master_table"] = list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            save_dict["common"] = list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            save_dict["classified"] = list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value] = save_dict
        else:
            tss_by_type[tss_type.value]["master_table"] += list(master_table_filtered_forward["Pos"].values) + list(master_table_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["common"] += list(common_filtered_forward["Pos"].values) + list(common_filtered_reverse["Pos"].values)
            tss_by_type[tss_type.value]["classified"] += list(classified_filtered_forward["Pos"].values) + list(classified_filtered_reverse["Pos"].values)

In [ ]:
performance_dict = {}
for tss_type in cs.TSSType:
    tss_by_type[tss_type.value]["master_table"] = set(tss_by_type[tss_type.value]["master_table"])
    tss_by_type[tss_type.value]["common"] = set(tss_by_type[tss_type.value]["common"])
    tss_by_type[tss_type.value]["classified"] = set(tss_by_type[tss_type.value]["classified"])

    common = len(tss_by_type[tss_type.value]["common"])
    classified = len(tss_by_type[tss_type.value]["classified"])
    master_table = len(tss_by_type[tss_type.value]["master_table"])
    
    performance_dict[tss_type.value] = {"precision": precision(common, classified), "recall": recall(common, master_table), "jaccard": jaccard(common, classified, master_table)}

In [ ]:
performances_full["bacteroides plasmid"] = performance_dict

In [ ]:
categories = [tss_type.value for tss_type in cs.TSSType]

In [ ]:
datasets = list(performances_full.keys())

In [ ]:
precision_data = {}
recall_data = {}
for category in categories:
    category_acc_precision = []
    category_acc_recall = []
    for dataset in datasets:
        category_acc_precision += [performances_full[dataset][category]["precision"]]
        category_acc_recall += [performances_full[dataset][category]["recall"]]
    precision_data[category] = category_acc_precision
    recall_data[category] = category_acc_recall

In [ ]:
import pandas as pd

In [ ]:
precision_df = pd.DataFrame(precision_data, index=datasets)
recall_df = pd.DataFrame(recall_data, index=datasets)

In [ ]:
# Number of categories
N = len(categories)

In [ ]:
from math import pi
import matplotlib.pyplot as plt

In [ ]:
# Function to create radar chart
def create_radar_chart(ax, precision, recall, title):
    # Compute angle for each category
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Prepare data for plotting
    precision_values = precision.tolist()
    recall_values = recall.tolist()
    
    precision_values += precision_values[:1]
    recall_values += recall_values[:1]
    
    # Plot precision
    ax.plot(angles, precision_values, label='Precision', linewidth=2, linestyle='solid', color='blue')
    ax.fill(angles, precision_values, alpha=0.25, color='blue')

    # Plot recall
    ax.plot(angles, recall_values, label='Recall', linewidth=2, linestyle='solid', color='green')
    ax.fill(angles, recall_values, alpha=0.25, color='green')

    # Add category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)

    # Set radial limits
    ax.set_ylim(0, 1)

    # Set the title and legend
    ax.set_title(title, size=12, color='black', y=1.1)
    ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.13))

In [ ]:
# Create a figure with subplots
fig, axs = plt.subplots(2, 2, figsize=(8, 8), subplot_kw=dict(polar=True))
axs = axs.flatten()  # Flatten the 2x2 array of axes for easy iteration

# Plot radar charts for each dataset
for i, dataset in enumerate(datasets):
    create_radar_chart(axs[i], precision_df.loc[dataset], recall_df.loc[dataset], dataset)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig('Radar_charts.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)

ax.scatter(sample_size[0:3]+sample_size[5:7], prediction_time[0:3]+prediction_time[5:7], color='blue')

ax.set_xscale('linear')
ax.set_yscale("linear")

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

sample_sizes = np.array(sample_size)
runtimes = np.array(prediction_time)

def linear(x, a, b):
    return a * x + b

popt_linear, _ = curve_fit(linear, sample_sizes, runtimes)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xscale('linear')

from sklearn.metrics import r2_score

r2 = r2_score(runtimes, linear(sample_sizes, *popt_linear))

# Plot the data and the fit
plt.scatter(sample_sizes, runtimes, label='Samples', color='black')
plt.plot([32840,6255429], linear(np.array([32840,6255429]), *popt_linear), label=f'Fit: linear\n$R^2 = {r2:.4f}$', color='green')

ax.set_xscale('log')
ax.set_yscale('log')

plt.title('TSSplorer Runtime vs. Sample Size')
plt.xlabel('Sample Size (bp)')
plt.ylabel('Runtime (seconds)')
plt.legend()
plt.grid(True)
plt.savefig('Runtime.png')
plt.show()
